In [10]:
!pip install transformers


In [13]:
from transformers import TFAutoModelForImageClassification, ViTFeatureExtractor
import tensorflow as tf
import numpy as np
import pandas as pd
from PIL import Image

from google.cloud import storage
import tensorflow as tf
import numpy as np
import pandas as pd
from PIL import Image
import os
import io


In [3]:
# 1. Authenticate Google Cloud
from google.colab import auth
auth.authenticate_user()
print("✅ Authenticated with Google Cloud")


✅ Authenticated with Google Cloud


In [14]:
from transformers import TFAutoModelForImageClassification, ViTFeatureExtractor

# ✅ Path to your saved Hugging Face model folder (contains tf_model.h5, config.json, etc.)
MODEL_PATH = "/content/drive/MyDrive/saved_vit_model-20250409T132041Z-001/saved_vit_model"

# ✅ Path to your recommendation CSV file
CSV_PATH = "/content/drive/MyDrive/aurora_products_B.csv"

# ✅ Load model and feature extractor
vit_model = TFAutoModelForImageClassification.from_pretrained(MODEL_PATH)
feature_extractor = ViTFeatureExtractor.from_pretrained(MODEL_PATH)

# ✅ Labels from the trained config
CATEGORIES = list(vit_model.config.id2label.values())
CRITICAL_CONDITIONS = ["Acne", "Milia", "Keratosis", "Hyperpigmentation"]

# ✅ Load CSV
import pandas as pd
df = pd.read_csv(CSV_PATH)


All model checkpoint layers were used when initializing TFViTForImageClassification.

All the layers of TFViTForImageClassification were initialized from the model checkpoint at /content/drive/MyDrive/saved_vit_model-20250409T132041Z-001/saved_vit_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFViTForImageClassification for predictions without further training.
/usr/local/lib/python3.11/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [19]:
# Manual label mapping from model's raw output labels to real disease names
LABEL_TO_NAME = {
    "LABEL_0": "Acne",
    "LABEL_1": "Milia",
    "LABEL_2": "Hyperpigmentation",
    "LABEL_3": "Wrinkles",
    "LABEL_4": "Keratosis",
    "LABEL_5": "Oily Skin",
    "LABEL_6": "Dry Skin",
    "LABEL_7": "Normal",
    "LABEL_8": "Non-Wrinkled Skin"
}

def predict_skin_condition(image_file):
    image = Image.open(image_file).convert("RGB").resize((224, 224))
    inputs = feature_extractor(images=[np.array(image)], return_tensors='tf')

    logits = vit_model(**inputs).logits
    probs = tf.nn.softmax(logits, axis=1).numpy()[0]

    top_idx = np.argmax(probs)
    confidence = float(probs[top_idx])
    raw_label = CATEGORIES[top_idx]
    condition = LABEL_TO_NAME.get(raw_label, raw_label)

    result = {
        "condition": condition,
        "confidence": round(confidence, 4),
    }

    matches = df[df['Targets'].str.lower().str.contains(condition.lower(), na=False)]
    recommendations = matches.head(3).to_dict(orient='records')

    if confidence >= 0.99:
        result["recommendation_type"] = "products"
        result["recommendations"] = recommendations
    elif confidence < 0.90 and condition in CRITICAL_CONDITIONS:
        result["recommendation_type"] = "refer"
        result["message"] = "Model is not confident and condition is critical. Please consult a dermatologist."
        result["recommendations"] = recommendations
    else:
        result["recommendation_type"] = "cautious_products"
        result["message"] = "Model is moderately confident. Use recommended products with care."
        result["recommendations"] = recommendations

    return result


In [20]:
from google.colab import files
uploaded = files.upload()  # Upload your skin image

for filename in uploaded:
    with open(filename, 'rb') as img_file:
        result = predict_skin_condition(img_file)
        print("📋 Prediction Result:")
        print(result)


Saving download (4).jpeg to download (4) (2).jpeg
📋 Prediction Result:
{'condition': 'Acne', 'confidence': 0.735, 'recommendation_type': 'refer', 'message': 'Model is not confident and condition is critical. Please consult a dermatologist.', 'recommendations': [{'Product': 'Lavender Foaming Face Wash', 'Suitable for': 'Sensitive Skin, Acne-prone Skin, Texture, Teen Age, Fine Lines, Wrinkles, Dullness, Environmental Damage, Irritation, Normal skin, Combination skin, Oily skin, Dry skin', 'Targets': 'Dull skin, Uneven skin tone, Redness, Broken & Acne Skin, Enlarged pores, Hyperpigmentation', 'When to apply': 'AM PM'}, {'Product': 'Charcoal Detox Soap', 'Suitable for': 'Normal skin, Combination skin, Oily skin', 'Targets': 'Uneven skin tone, Broken & Acne Skin, Enlarged pores, Hyperpigmentation', 'When to apply': 'AM, PM'}, {'Product': nan, 'Suitable for': 'Normal Skin , Dry Skin , Oily Skin, Sensitive Skin, Acne-Prone Skin , Combination Skin', 'Targets': 'Dark spots & pigmentation, , Un

In [ ]:
from google.colab import auth
auth.authenticate_user()

from google.cloud import storage

client = storage.Client()
bucket = client.bucket("aurora-project")

# Upload model files one by one
model_path = "/content/drive/MyDrive/saved_vit_model/saved_vit_model"
blob = bucket.blob("models/vit_model/config.json")
blob.upload_from_filename(f"{model_path}/config.json")

blob = bucket.blob("models/vit_model/tf_model.h5")
blob.upload_from_filename(f"{model_path}/tf_model.h5")

blob = bucket.blob("models/vit_model/preprocessor_config.json")
blob.upload_from_filename(f"{model_path}/preprocessor_config.json")

# Upload CSV
blob = bucket.blob("data/aurora_products_cleaned.csv")
blob.upload_from_filename("/content/drive/MyDrive/aurora_products_B.csv")


In [27]:
import requests
from google.colab import files

uploaded = files.upload()
file_name = next(iter(uploaded))

url = "  https://us-central1-aurora-457407.cloudfunctions.net/predict"
with open(file_name, "rb") as img:
    response = requests.post(url, files={'file': img})

print("✅ Status Code:", response.status_code)
print("📄 Raw Response:\n", response.text)


Saving Acne_5.jpg to Acne_5 (2).jpg
✅ Status Code: 200
📄 Raw Response:
 {"condition":"Milia","confidence":0.5863,"message":"Model is not confident and condition is critical. Please consult a dermatologist.","recommendation_type":"refer","recommendations":[]}

